# LTO RAG Table of Contents

1. Import Libraries
2. Connect to Ollama Server
3. Ingestion
4. QA Generation
5. Embedding and Retrieval<br>
 **5.A** Dense via FAISS  
 **5.B** FAISS Retrieval Evaluator  
 **5.C** FAISS Retrieval Evaluation  
 **5.D** Sparse Embedding via BM25  
 **5.E** Hybrid Embedding via Reciprocal Rank Fusion  
 **5.F** Hybrid Retrieval Evaluator  
 **5.G** Hybrid Retrieval Evaluation  
6. Post Retrieval<br>
 **6.A** Summarization  
 **6.B** Evaluation Generation  
7. Querying<br>
 **7.A** Query Transforms  
8. Query Generation
9. TDC Exam Evaluation
10. Similarity Evaluation
11. Relevancy Evaluation


# 1. Import Libraries

In [1]:
import os
import fitz
import re

from ollama import Client
import faiss
import pandas as pd
import numpy as np
import Stemmer
from tqdm import tqdm
import gradio as gr
import json

from llama_index.core import Document
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.retrievers import BaseRetriever, QueryFusionRetriever
from llama_index.core.schema import TextNode, NodeWithScore
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.llms.ollama import Ollama


# For QA generation
import uuid
import warnings
from typing import Dict, List, Optional, Tuple

from llama_index.core.bridge.pydantic import BaseModel
from llama_index.core.llms.utils import LLM
from llama_index.core.schema import MetadataMode, TextNode
from llama_index.core.settings import Settings


# 2. Connect to Ollama Server

In [2]:
client = Client(
  host='http://localhost:11434',
)

# 3. Ingestion

In [3]:
# Path to the dataset folder
DATASET_PATH = r'/home/jeryl4913/lto_rag_reviewer/notebooks/extracted_text.json'

def get_text_and_metadata(input_path):
    """Load text and metadata from a file and perform chunking."""
    with open(input_path, "r", encoding="utf-8") as f:
        extracted_data = json.load(f)

    texts = []
    metadata = []

    for entry in tqdm(extracted_data, desc="Processing entries"):
        text = entry["text"]
        source_metadata = {
            "source": entry["source"],
            "folder": entry["folder"],
            "file_name": entry["file_name"],
            "page": entry["page"],
            "title": entry["title"],
            "url": entry["url"]
        }
        texts.append(text)
        metadata.append(source_metadata)

    return texts, metadata


In [4]:
docs, metadatas = get_text_and_metadata(DATASET_PATH)

Processing entries: 100%|██████████| 7815/7815 [00:00<00:00, 814736.67it/s]


In [5]:
documents = [Document(text=docs[t], metadata=metadatas[t]) for t in range(len(docs))]
splitter = TokenTextSplitter(
    chunk_size=512,
    chunk_overlap=20,
    separator=" ",
)
nodes = splitter.get_nodes_from_documents(documents)

In [6]:
print(nodes[0])

Node ID: fd0f9b7a-48f0-4776-b789-e15d93319f12
Text: Land Transportation Office (LTO) The concept of land
transportation system in the Philippines started when our ancestors
invented the means of locomotion with the animals in moving people and
goods from place to place. Although the means of land transportation
during the early days were not as sophisticated as the modern vehicles
of today and th...


# 4. QA Generation
Used for Retreival Evaluation:
1. Get all document nodes
2. Generate question for each node (using llama 3.2)
3. Question Answer pairs: Generated Question, Node text

In [7]:
"""Common utils for embeddings."""

import json
import re
import uuid
import warnings
from typing import Dict, List, Optional, Tuple

from llama_index.core.bridge.pydantic import BaseModel
from llama_index.core.llms.utils import LLM
from llama_index.core.schema import MetadataMode, TextNode
from llama_index.core.settings import Settings
from tqdm import tqdm


class EmbeddingQAFinetuneDataset(BaseModel):
    """Embedding QA Finetuning Dataset.

    Args:
        queries (Dict[str, str]): Dict id -> query.
        corpus (Dict[str, str]): Dict id -> string.
        relevant_docs (Dict[str, List[str]]): Dict query id -> list of doc ids.

    """

    queries: Dict[str, str]  # dict id -> query
    corpus: Dict[str, str]  # dict id -> string
    relevant_docs: Dict[str, List[str]]  # query id -> list of doc ids
    mode: str = "text"

    @property
    def query_docid_pairs(self) -> List[Tuple[str, List[str]]]:
        """Get query, relevant doc ids."""
        return [
            (query, self.relevant_docs[query_id])
            for query_id, query in self.queries.items()
        ]

    def save_json(self, path: str) -> None:
        """Save json."""
        with open(path, "w") as f:
            json.dump(self.model_dump(), f, indent=4)

    @classmethod
    def from_json(cls, path: str) -> "EmbeddingQAFinetuneDataset":
        """Load json."""
        with open(path) as f:
            data = json.load(f)
        return cls(**data)


DEFAULT_QA_GENERATE_PROMPT_TMPL = """
Context information is below.

---------------------
{context_str}
---------------------

Based on the above context, generate {num_questions_per_chunk} diverse and specific questions that focus on key details and concepts. \
Do not include prefatory phrases like 'Here are some questions'. Directly list the questions in numerical order.
"""


# generate queries as a convenience function
def generate_qa_embedding_pairs(
    nodes: List[TextNode],
    llm: Optional[LLM] = None,
    qa_generate_prompt_tmpl: str = DEFAULT_QA_GENERATE_PROMPT_TMPL,
    num_questions_per_chunk: int = 2,
) -> EmbeddingQAFinetuneDataset:
    """Generate examples given a set of nodes."""
    llm = llm or Settings.llm
    node_dict = {
        node.node_id: node.get_content(metadata_mode=MetadataMode.NONE)
        for node in nodes
    }

    queries = {}
    relevant_docs = {}
    for node_id, text in tqdm(node_dict.items()):
        query = qa_generate_prompt_tmpl.format(
            context_str=text, num_questions_per_chunk=num_questions_per_chunk
        )
        response = llm.complete(query)

        result = str(response).strip().split("\n")
        questions = [
            re.sub(r"^\d+[\).\s]", "", question).strip() for question in result
        ]
        questions = [
            question
            for question in questions
            if len(question) > 0 and question.endswith("?") and "question" not in question.lower()
        ][:num_questions_per_chunk]

        num_questions_generated = len(questions)
        if num_questions_generated < num_questions_per_chunk:
            warnings.warn(
                f"Fewer questions generated ({num_questions_generated}) "
                f"than requested ({num_questions_per_chunk}) for node {node_id}."
            )

        for question in questions:
            question_id = str(uuid.uuid4())
            queries[question_id] = question
            relevant_docs[question_id] = [node_id]

    # construct dataset
    return EmbeddingQAFinetuneDataset(
        queries=queries, corpus=node_dict, relevant_docs=relevant_docs
    )


In [8]:
import os

access_token = os.popen("echo $OPENAI_API_KEY").read().strip()


if access_token is None:
    raise ValueError("OpenAI API key not found in environment.")

#print(access_token)


In [9]:
from llama_index.core.evaluation import EmbeddingQAFinetuneDataset
from llama_index.llms.ollama import Ollama
from llama_index.llms.openai import OpenAI

# Instantiate LLM
ollama_llm = Ollama(model="llama3.3", request_timeout=300)

# llm = OpenAI(model="gpt-4", access_token=access_token)


# Example usage of the generate_qa_embedding_pairs
qa_dataset = generate_qa_embedding_pairs(
    nodes[:60], llm=ollama_llm, num_questions_per_chunk=1
)

queries = qa_dataset.queries.values()
print(list(queries)[2])

# Save the dataset
qa_dataset.save_json("pg_eval_dataset_LLAMA3.3_generated.json")
print("Successfully created QA dataset")


100%|██████████| 60/60 [18:17<00:00, 18.30s/it]

What significant change occurred to the Land Transportation Commission as a result of the creation of the Ministry of Transportation and Communications through Executive Order Number 546 in 1979?
Successfully created QA dataset


In [10]:
# Load
qa_dataset = EmbeddingQAFinetuneDataset.from_json("pg_eval_dataset_LLAMA3.3_generated.json")
print("Successfully loaded QA dataset")

Successfully loaded QA dataset


# 5. Embedding and Retrieval

## 5.A. Dense via FAISS

In [11]:
def generate_embeddings(nodes, client, model):
    # Generate embeddings for documents using Ollama
    for doc in tqdm(nodes):
        response = client.embeddings(prompt=doc.text, model=model)
        doc.embedding = response["embedding"]
    return nodes

In [12]:
class FaissIndexer:
    """
    Faiss-based indexer for efficient similarity search using inner-product (cosine) similarity.

    This class handles the creation and management of a FAISS index from node embeddings.
    
    :ivar faiss_index: The FAISS index for storing and querying embeddings.
    :vartype faiss_index: faiss.IndexFlatIP
    :ivar embedding_dim: Dimensionality of the embeddings.
    :vartype embedding_dim: int
    """

    def __init__(self):
        """
        Initialize the FaissIndexer class.

        :ivar faiss_index: The FAISS index, initialized as None.
        :ivar embedding_dim: The dimension of embeddings, initialized as None.
        """
        self.faiss_index = None
        self.embedding_dim = None

    def normalize_embeddings(self, embeddings):
        """
        Normalize embeddings to have unit L2 norm.

        :param embeddings: Array of embeddings to normalize.
        :type embeddings: np.ndarray
        :return: Normalized embeddings.
        :rtype: np.ndarray
        """
        return embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

    def build_index(self, nodes):
        """
        Build the FAISS index from a list of nodes containing embeddings.

        :param nodes: List of nodes, where each node contains an `embedding` attribute.
        :type nodes: list
        :raises ValueError: If the nodes list is empty or embeddings are inconsistent.
        """
        if not nodes:
            raise ValueError("Nodes list cannot be empty.")
        
        embeddings = np.array([np.array(node.embedding) for node in nodes])
        normalized_embeddings = self.normalize_embeddings(embeddings)

        self.embedding_dim = normalized_embeddings[0].shape[0]
        self.faiss_index = faiss.IndexFlatIP(self.embedding_dim)  # Inner-product similarity
        self.faiss_index.add(normalized_embeddings)

    def get_index(self):
        """
        Get the FAISS index instance.

        :return: The FAISS index used for similarity search.
        :rtype: faiss.IndexFlatIP
        :raises ValueError: If the index has not been built.
        """
        if self.faiss_index is None:
            raise ValueError("Index has not been built yet. Call 'build_index' first.")
        return self.faiss_index

In [13]:
class FAISSVectorStoreRetriever(BaseRetriever):
    def __init__(self, faiss_index, documents):
        """
        Initialize the FAISS retriever.
        :param faiss_index: The FAISS index containing precomputed embeddings.
        :param documents: List of document chunks.
        :param embeddings: Precomputed embeddings corresponding to the document chunks.
        """
        self.faiss_index = faiss_index
        self.documents = documents

    def _retrieve(self, query_embedding, top_k=5):
        """
        Retrieve the top-k nearest neighbors using the FAISS index.
        :param query_embedding: The embedding of the query.
        :param top_k: Number of top results to retrieve.
        """

        norm_query_embedding = np.array([query_embedding])
        norm_query_embedding /= np.linalg.norm(norm_query_embedding, axis=1, keepdims=True)

        distances, indices = self.faiss_index.search(norm_query_embedding, top_k)
        retrieved_docs = [
            NodeWithScore(node=self.documents[idx], score=1 - dist)
            for idx, dist in zip(indices[0], distances[0])
            if idx != -1
        ]
        return retrieved_docs

## 5.B. FAISS Retreival Evaluator

In [14]:
from typing import List, Tuple, Any
from pydantic import Field, ConfigDict
from llama_index.core.evaluation.retrieval.base import (
    BaseRetrievalEvaluator,
    RetrievalEvalMode,
    RetrievalEvalResult
)
from llama_index.core.evaluation.retrieval.metrics import resolve_metrics
from llama_index.core.response.notebook_utils import display_source_node

class FAISSRetrievalEvaluator(BaseRetrievalEvaluator):
    retriever: "FAISSVectorStoreRetriever" = Field(..., description="FAISS Retriever instance")
    Print_Results: bool = Field(default=False, description="Whether to print retrieved results")

    model_config = ConfigDict(extra="forbid")  

    @classmethod
    def from_metric_names(
        cls,
        metric_names: List[str],
        retriever: "FAISSVectorStoreRetriever",
        Print_Results: bool = False,
        **kwargs: Any,
    ) -> "FAISSRetrievalEvaluator":
        metric_types = resolve_metrics(metric_names)
        metrics = [metric() for metric in metric_types]
        return cls(metrics=metrics, retriever=retriever, Print_Results=Print_Results, **kwargs)

    async def _aget_retrieved_ids_and_texts(
        self,
        query: str,
        mode: RetrievalEvalMode = RetrievalEvalMode.TEXT,
    ) -> Tuple[List[str], List[str]]:
        response = client.embeddings(prompt=query, model="mxbai-embed-large")
        query_embedding = response["embedding"]
        retrieved_docs = self.retriever._retrieve(query_embedding, top_k=15)
        
        # Conditionally print results
        if self.Print_Results:
            for doc in retrieved_docs:
                display_source_node(doc, source_length=1000)  # Directly use doc
        
        retrieved_ids = [doc.id_ for doc in retrieved_docs]  # doc, not doc.node
        retrieved_texts = [doc.text for doc in retrieved_docs]
        return retrieved_ids, retrieved_texts

    async def aevaluate(
        self,
        query: str,
        expected_ids: List[str],
        expected_texts: List[str] = [],
        mode: RetrievalEvalMode = RetrievalEvalMode.TEXT,
        **kwargs: Any,
    ) -> RetrievalEvalResult:
        retrieved_ids, retrieved_texts = await self._aget_retrieved_ids_and_texts(query, mode)
        metric_dict = {}
        
        for metric in self.metrics:
            # Call compute instead of evaluate
            result = metric.compute(
                query=query,
                expected_ids=expected_ids,
                retrieved_ids=retrieved_ids,
                expected_texts=expected_texts,
                retrieved_texts=retrieved_texts,
                **kwargs
            )
            metric_dict[metric.metric_name] = result  # Store the whole RetrievalMetricResult object
            print(f"{metric.metric_name}: {result.score}")  # Print each metric result
         
        # Return RetrievalEvalResult with all required fields
        return RetrievalEvalResult(
            query=query,
            retrieved_ids=retrieved_ids,
            retrieved_texts=retrieved_texts,  # Include retrieved texts
            expected_ids=expected_ids,         # Pass expected ids
            expected_texts=expected_texts,      # Include expected texts
            metric_dict=metric_dict            # Pass full RetrievalMetricResult objects
        )


In [15]:
nodes_embed = generate_embeddings(nodes,  client, "mxbai-embed-large")

100%|██████████| 12302/12302 [46:40<00:00,  4.39it/s]  


## 5.C. FAISS Retreival Evaluation

In [16]:
indexer = FaissIndexer()

indexer.build_index(nodes)  
faiss_index = indexer.get_index()

retriever = FAISSVectorStoreRetriever(faiss_index=indexer.get_index(), documents=nodes_embed)

metrics = ["hit_rate", "mrr", "precision", "recall", "ap", "ndcg"]
evaluator = FAISSRetrievalEvaluator.from_metric_names(
    metric_names=metrics,
    retriever=retriever,
    Print_Results=True
)


sample_id, sample_query = list(qa_dataset.queries.items())[1]
sample_expected = qa_dataset.relevant_docs[sample_id]

print("Sample Problem:")
print(f"sample id: {sample_id}, text: {sample_query}")
print(f"sample exp: {sample_expected}")

print("\nRetrieval results:")

result = await evaluator.aevaluate(
    query=sample_query,
    expected_ids=sample_expected,
)

print("\n")
print(result)

Sample Problem:
sample id: adec2f94-cf36-48ab-8bcb-8bad96ad63ce, text: What significant event occurred on June 20, 1964, that affected the Motor Vehicle Office and what new entity was created as a result of this event?
sample exp: ['0e47f583-a3d5-4bac-8b8e-77bba34385de']

Retrieval results:


**Node ID:** 0e47f583-a3d5-4bac-8b8e-77bba34385de<br>**Similarity:** 0.3204209804534912<br>**Text:** Motor Vehicles. The Chief of the Division was called the Superintendent of Division of Motor Vehicles. Act No. 3992 was amended by Commonwealth Act Numbers 123, 548, 556, 652 and Republic Act Numbers 314, 587, and 2383. On June 2, 1945, Department Order No. 4 was issued by the Department of Public Works and Highways reorganizing the Division. This took effect after the liberation of the Philippines from the Japanese invasion. In 1947, Executive Order No. 94 was promulgated reorganizing the different executive departments, bureaus and offices. Under Section 82 of this E.O., the Division of Motor Vehicles was upgraded into the Motor Vehicles Office (MVO) with the category of a Bureau. The Chief of the MVO enjoyed the rights and privileges of a Bureau Director. During the fifties and early sixties, our country started undergoing rapid economic development. Industrialization advanced and as a consequence, more and better roads were constructed. The Filipino then realized the need for mo...<br>

**Node ID:** 27fc22a5-edfe-4bb2-b31b-5c93b88ac06f<br>**Similarity:** 0.33813750743865967<br>**Text:** crucial role in overseeing and implementing the regulations outlined in RA 3045.Evolution into Division of Motor Vehicles:Subsequent amendments, notably Act No. 3992 in 1933, saw the transformation of the Automobile Division into the Division of Motor Vehicles. This rebranding reflected a more specialized and refined approach to motor vehicle governance. It was later replaced by the Republic<br>

**Node ID:** a80209bd-ed88-4dbd-b984-2fc61ec51f80<br>**Similarity:** 0.3570370674133301<br>**Text:** was originally established and governed by law, thanks to Executive Order (EO) 287. Under this law, all motor vehicles purchased or owned by any department, bureau, office, division, branch, or unit thereof, or of any agency or instrumentality of the Government, including those of the<br>

**Node ID:** 5b64f487-370c-4b61-a6c8-007b184d4bfd<br>**Similarity:** 0.3733302354812622<br>**Text:** WELGOME TO THE LTO INTERNAL PORTAL  Click on Motor Vehicle BB reronetesveyone + Click on Create + R JUN 13 2004 MWNreisTeRe =e [<br>

...

**Node ID:** f8744476-a6ef-4dbe-85c1-ee275cb23384<br>**Similarity:** 0.3963847756385803<br>**Text:** .. personnel using the DO MV Maintenance Facility upon receipt of the latest copy of Official Receipt/Certificate of Registration from the customer. 3. If the registration date is different from the assigned plate ending of the vehicle, the Next Registration Date field shall be updated. This field shall automatically be populated by the system when the Last Registration Date is updated by the user and shall remain enabled for editing as deemed necessary. 4. For registration renewal transactions not requiring the collection of Change Venue Fee, the Last District Office must be updated by the user based on the last venue of registration stated in the latest copy of the OR. This memorandum shall take effect on 14 November 2022. All orders/memoranda inconsistent herewith are hereby repealed accordingly. For strict and immediate compliance. -1.l t:. ~ AT~4~0FILO E. G Assistant Secretary IZ III, CESO V r- U.P. i,.AW CENTER - 1 OFFICE el tht NATIONll.l AC'MIN\Sl RWV.E REG~TER\ M1111nlw a11...<br>

**Node ID:** 2d4fefe2-fcd1-4999-9219-4ed3ca7cd73a<br>**Similarity:** 0.40047669410705566<br>**Text:** Vehicle (MV)Renewal of Motor Vehicle (MV) RegistrationStorage of Motor VehicleMiscellaneous TransactionsRe-Stamping of Engine/Chasis NumberRequest for Motor Vehicle VerificationMotor Vehicle InspectionMiscellaneous TransactionsDuplicate OR/CRDuplicate PlateTransfer of OwnershipAnnotation & Cancellation of MortgageRevision of RecordsDuplicate License (for Lost Licenses)InsuranceProper Driver EducationTheoretical Driving Course (TDC) and<br>

**Node ID:** 5759dda6-6bef-4b2e-b3bf-a8d0ae5afb99<br>**Similarity:** 0.4016345739364624<br>**Text:** Registration of Motor Vehicle (MV)Renewal of Motor Vehicle (MV) RegistrationStorage of Motor VehicleMiscellaneous TransactionsRe-Stamping of Engine/Chassis NumberRequest for Motor Vehicle VerificationMotor Vehicle InspectionMiscellaneous TransactionsDuplicate OR/CRDuplicate PlateTransfer of OwnershipAnnotation & Cancellation of MortgageRevision of RecordsDuplicate License (for Lost Licenses)InsuranceProper Driver<br>

**Node ID:** d58a47da-d2ac-446b-ab44-39a5593e5909<br>**Similarity:** 0.4016345739364624<br>**Text:** Registration of Motor Vehicle (MV)Renewal of Motor Vehicle (MV) RegistrationStorage of Motor VehicleMiscellaneous TransactionsRe-Stamping of Engine/Chassis NumberRequest for Motor Vehicle VerificationMotor Vehicle InspectionMiscellaneous TransactionsDuplicate OR/CRDuplicate PlateTransfer of OwnershipAnnotation & Cancellation of MortgageRevision of RecordsDuplicate License (for Lost Licenses)InsuranceProper Driver<br>

hit_rate: 1.0
mrr: 1.0
precision: 0.06666666666666667
recall: 1.0
ap: 1.0
ndcg: 1.0


Query: What significant event occurred on June 20, 1964, that affected the Motor Vehicle Office and what new entity was created as a result of this event?
Metrics: {'hit_rate': 1.0, 'mrr': 1.0, 'precision': 0.06666666666666667, 'recall': 1.0, 'ap': 1.0, 'ndcg': 1.0}



In [17]:
  
evaluator = FAISSRetrievalEvaluator.from_metric_names(
    metric_names=metrics,
    retriever=retriever,
    Print_Results=False
)

eval_results = await evaluator.aevaluate_dataset(qa_dataset)


def display_results(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    columns = {
        "retrievers": [name],
        **{k: [full_df[k].mean()] for k in metrics},
    }

    metric_df = pd.DataFrame(columns)

    return metric_df


display_results("top-2 eval", eval_results)

hit_rate: 1.0
mrr: 1.0
precision: 0.06666666666666667
recall: 1.0
ap: 1.0
ndcg: 1.0


hit_rate: 1.0
mrr: 1.0
precision: 0.06666666666666667
recall: 1.0
ap: 1.0
ndcg: 1.0
hit_rate: 1.0
mrr: 1.0
precision: 0.06666666666666667
recall: 1.0
ap: 1.0
ndcg: 1.0
hit_rate: 1.0
mrr: 1.0
precision: 0.06666666666666667
recall: 1.0
ap: 1.0
ndcg: 1.0
hit_rate: 0.0
mrr: 0.0
precision: 0.0
recall: 0.0
ap: 0.0
ndcg: 0.0
hit_rate: 0.0
mrr: 0.0
precision: 0.0
recall: 0.0
ap: 0.0
ndcg: 0.0
hit_rate: 1.0
mrr: 1.0
precision: 0.06666666666666667
recall: 1.0
ap: 1.0
ndcg: 1.0
hit_rate: 1.0
mrr: 1.0
precision: 0.06666666666666667
recall: 1.0
ap: 1.0
ndcg: 1.0
hit_rate: 1.0
mrr: 0.2
precision: 0.06666666666666667
recall: 1.0
ap: 0.2
ndcg: 0.38685280723454163
hit_rate: 1.0
mrr: 0.25
precision: 0.06666666666666667
recall: 1.0
ap: 0.25
ndcg: 0.43067655807339306
hit_rate: 1.0
mrr: 0.06666666666666667
precision: 0.06666666666666667
recall: 1.0
ap: 0.06666666666666667
ndcg: 0.25
hit_rate: 0.0
mrr: 0.0
precision: 0.0
recall: 0.0
ap: 0.0
ndcg: 0.0
hit_rate: 1.0
mrr: 0.3333333333333333
precision: 0.066666

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,top-2 eval,0.733333,0.419518,0.048889,0.733333,0.419518,0.491658


In [18]:
#indexing
index = FaissIndexer()
index.build_index(nodes_embed)
faiss_index = index.get_index()

faiss_retriever = FAISSVectorStoreRetriever(faiss_index=faiss_index,documents=nodes_embed)

## 5.D Sparse Embedding via BM25

In [19]:
bm25_retriever = BM25Retriever.from_defaults(
   nodes=nodes,
   similarity_top_k=5,
   stemmer=Stemmer.Stemmer("english"),
   language="english",
)

## 5.E Hybrid Retrieval via Reciprocal Rank

In [20]:
def hybrid_embedding(results: dict, top_k: int):
    x = QueryFusionRetriever
    ranked_results = QueryFusionRetriever._reciprocal_rerank_fusion(x, results)
    return ranked_results[:top_k]

## 5.F Hybrid Retrieval Evaluator

In [21]:
from typing import List, Tuple, Any, Dict
from pydantic import Field, ConfigDict
from llama_index.core.evaluation.retrieval.base import (
    BaseRetrievalEvaluator,
    RetrievalEvalMode,
    RetrievalEvalResult
)
from llama_index.core.evaluation.retrieval.metrics import resolve_metrics
from llama_index.core.response.notebook_utils import display_source_node

class HybridRetrievalEvaluator(BaseRetrievalEvaluator):
    faiss_retriever: "FAISSVectorStoreRetriever" = Field(..., description="FAISS Retriever instance")
    bm25_retriever: "BM25Retriever" = Field(..., description="BM25 Retriever instance")
    Print_Results: bool = Field(default=False, description="Whether to print retrieved results")

    model_config = ConfigDict(extra="forbid")  

    @classmethod
    def from_metric_names(
        cls,
        metric_names: List[str],
        faiss_retriever: "FAISSVectorStoreRetriever",
        bm25_retriever: "BM25Retriever",
        Print_Results: bool = False,
        **kwargs: Any,
    ) -> "HybridRetrievalEvaluator":
        metric_types = resolve_metrics(metric_names)
        metrics = [metric() for metric in metric_types]
        return cls(metrics=metrics, faiss_retriever=faiss_retriever, bm25_retriever=bm25_retriever, Print_Results=Print_Results, **kwargs)

    async def _aget_retrieved_ids_and_texts(
        self,
        query: str,
        mode: RetrievalEvalMode = RetrievalEvalMode.TEXT,
        top_k: int = 15
    ) -> Tuple[List[str], List[str]]:
        response = client.embeddings(prompt=query, model="mxbai-embed-large")
        query_embedding = response["embedding"]

        # FAISS retrieval
        faiss_docs = self.faiss_retriever._retrieve(query_embedding, top_k=top_k)

        # BM25 retrieval
        bm25_docs = self.bm25_retriever.retrieve(query)

        # Combine results
        results = {'faiss': faiss_docs, 'bm25': bm25_docs}
        ranked_results = QueryFusionRetriever._reciprocal_rerank_fusion(None, results)
        ranked_results = ranked_results[:top_k]

        # Optionally display results
        if self.Print_Results:
            for doc in ranked_results:
                display_source_node(doc, source_length=1000)

        retrieved_ids = [doc.id_ for doc in ranked_results]
        retrieved_texts = [doc.text for doc in ranked_results]
        return retrieved_ids, retrieved_texts

    async def aevaluate(
        self,
        query: str,
        expected_ids: List[str],
        expected_texts: List[str] = [],
        mode: RetrievalEvalMode = RetrievalEvalMode.TEXT,
        **kwargs: Any,
    ) -> RetrievalEvalResult:
        retrieved_ids, retrieved_texts = await self._aget_retrieved_ids_and_texts(query, mode)
        metric_dict = {}
        
        for metric in self.metrics:
            # Call compute instead of evaluate
            result = metric.compute(
                query=query,
                expected_ids=expected_ids,
                retrieved_ids=retrieved_ids,
                expected_texts=expected_texts,
                retrieved_texts=retrieved_texts,
                **kwargs
            )
            metric_dict[metric.metric_name] = result  # Store the whole RetrievalMetricResult object
            print(f"{metric.metric_name}: {result.score}")  # Print each metric result
        
        # Return RetrievalEvalResult with all required fields
        return RetrievalEvalResult(
            query=query,
            retrieved_ids=retrieved_ids,
            retrieved_texts=retrieved_texts,
            expected_ids=expected_ids,
            expected_texts=expected_texts,
            metric_dict=metric_dict
        )


## 5.G BM25 Retreival Evaluation

In [22]:
evaluator = HybridRetrievalEvaluator.from_metric_names(
    metric_names=metrics,
    faiss_retriever=faiss_retriever,
    bm25_retriever=bm25_retriever,
    Print_Results=True  # Toggle as needed
)

sample_id, sample_query = list(qa_dataset.queries.items())[1]
sample_expected = qa_dataset.relevant_docs[sample_id]

print("Sample Problem:")
print(f"sample id: {sample_id}, text: {sample_query}")
print(f"sample exp: {sample_expected}")

print("\nRetrieval results:")

result = await evaluator.aevaluate(
    query=sample_query,
    expected_ids=sample_expected
)

print("\n")
print(result)

Sample Problem:
sample id: adec2f94-cf36-48ab-8bcb-8bad96ad63ce, text: What significant event occurred on June 20, 1964, that affected the Motor Vehicle Office and what new entity was created as a result of this event?
sample exp: ['0e47f583-a3d5-4bac-8b8e-77bba34385de']

Retrieval results:


**Node ID:** 5759dda6-6bef-4b2e-b3bf-a8d0ae5afb99<br>**Similarity:** 0.016666666666666666<br>**Text:** Registration of Motor Vehicle (MV)Renewal of Motor Vehicle (MV) RegistrationStorage of Motor VehicleMiscellaneous TransactionsRe-Stamping of Engine/Chassis NumberRequest for Motor Vehicle VerificationMotor Vehicle InspectionMiscellaneous TransactionsDuplicate OR/CRDuplicate PlateTransfer of OwnershipAnnotation & Cancellation of MortgageRevision of RecordsDuplicate License (for Lost Licenses)InsuranceProper Driver<br>

**Node ID:** 32a21628-5e32-48c6-a152-90a5732b22b2<br>**Similarity:** 0.016666666666666666<br>**Text:** people, or objects on or beside the road. It usually results in vehicular damage, physical injuries, or even fatalities. They are unfortunate events that can happen to anyone while driving on the road. Knowing what to do in cases of car accidents is crucial because it ensures a proper and swift response to mitigate further risks and address the immediate aftermath effectively.Being prepared with the necessary knowledge and steps to take during a car accident can make a significant difference in ensuring everyone?s safety and resolving any resulting issues promptly. It helps individuals involved in such incidents navigate through the complex process of reporting, documenting, and potentially seeking assistance from authorities or insurance providers.<br>

**Node ID:** d58a47da-d2ac-446b-ab44-39a5593e5909<br>**Similarity:** 0.01639344262295082<br>**Text:** Registration of Motor Vehicle (MV)Renewal of Motor Vehicle (MV) RegistrationStorage of Motor VehicleMiscellaneous TransactionsRe-Stamping of Engine/Chassis NumberRequest for Motor Vehicle VerificationMotor Vehicle InspectionMiscellaneous TransactionsDuplicate OR/CRDuplicate PlateTransfer of OwnershipAnnotation & Cancellation of MortgageRevision of RecordsDuplicate License (for Lost Licenses)InsuranceProper Driver<br>

**Node ID:** b19251e5-d840-43f2-add7-058858ad182d<br>**Similarity:** 0.01639344262295082<br>**Text:** Can You Use Commemorative Plates in the Philippines?The use of commemorative plates on motor vehicles in the Philippines is subject to a set of rules that ensures both respect for tradition and adherence to order?theLand Transportation Office (LTO)Administrative Order AOL-2010-021. Failure to adhere to these guidelines makes it an illegal use of commemorative plates. Without the appropriate permission or authorization to use of commemorative plates, doing so is also prohibited in the Philippines.This LTO AOL-2010-021 outlines the general guidelines on the use of commemorative plates?from the proper placement to the issuance and duration of commemorative plates, intricately weaving a connection between national, historical, and international significance with the nation?s roads. It also answers the question about whether commemorative plates are allowed for use in the Philippines or not.Table of ContentsToggleWhat are Commemorative PlatesWhat are Commemorative Plates forGuidelines fo...<br>

...

**Node ID:** b6855080-99d2-40a9-b23f-01485f9756fa<br>**Similarity:** 0.015151515151515152<br>**Text:** Registration of Motor Vehicle<br>

**Node ID:** 1e7fae0e-d2f5-41c8-a616-a7fd239b964d<br>**Similarity:** 0.014925373134328358<br>**Text:** ~1 (=?~ L---- ~ ~ " "' ,--.,_ J:- v 'i -o ' c -- .t s.. ~ .: ~ :. ~ - ~ ..-:  1. ACCREDITATION OF MANUFACTURERS, ASSEMBLERS, IMPORTERS, REBU ILDERS AND/OR DEALERS (MAJRDs) - NEW = ~ :~ ; '----' rd = w.. " * ~ An authority granted to MAIRDs to transact business with L TO Operations Division, L TO Central Office and Regional Office "  c - 1- -. "' r--~ .,, g ~ / ::.-::? v~ - '<., :.,,_; ~ f ;.. - <. ~ <; rn .;; ~ "" __..,, C> ~ ;:;, r- "' ~~:;-:~ ~.'\~ : ~-:t: "f~  . ;: .. .=#~ r  . __ ._; ,, (. - , .. _, c:::. ".:. LV t /;:; :77" ' ~); ;:: !:; ... : ~ {.~ .- ;, :. : -~ .,,..  ~ ... . '  't ~ :j c, :_  ~ :.~;:: ,, Highly Technical G28 - Government to Business Manufacturers, Assemblers, Importers, Rebuilders, and/or Dealers of Motor Vehicles and/or components (Any natural person who is at least 18 yea rs of age or any juridical person who is not disqualified by any existing law or regulation to engage in the manufacturing, assembly, importation,! sale and rebui lding, dealership of mot...<br>

**Node ID:** e96f523c-b634-4a2b-8200-0a896f8aa0dd<br>**Similarity:** 0.014705882352941176<br>**Text:** Motor Vehicles reflected a recognition of the need for more nuanced governance.Foundation for Ongoing RegulationsRA 3045 laid the foundation for subsequent regulations and adjustments in response to the evolving dynamics of motor vehicle use. Its principles continue to influence and guide contemporary efforts to ensure road safety, efficient traffic management, and responsible vehicle operation.Enduring LegacyThe enduring legacy of RA 3045 is reflected in the enduring commitment to structured and safe road governance. Its principles resonate in the ongoing efforts to address new challenges brought about by technological advancements and changes in transportation patterns.More than these benefits, the Republic Act No. 3045 has played a pivotal role in shaping the regulatory landscape of motor vehicles in the Philippines. Its influence is evident in the organized traffic systems, enhanced safety measures, and the establishment of foundational structures that persist in today?s road go...<br>

**Node ID:** bfc066f1-b2ec-43ed-831e-ee2d230776da<br>**Similarity:** 0.014492753623188406<br>**Text:** On June 2, 1980, Batas Pambansa Bilang 43 was passed providing for the issuance of permanent number plates to owners of motor vehicles and trailers, amending for the purpose section 17 of RA 4136. On March 20, 1985, Executive Order 1011 was promulgated. This Executive Order abolished the Board of Transportation and the Bureau of Land Transportation and established the Land Transportation Commission. The defunct BLT and BOT were merged and their powers, functions and responsibilities were transferred to the Land Transportation Commission (LTC) headed by a Chairman, assisted by four Commissioners. The LTC was tasked to perform functions such as registering motor vehicles, licensing of drivers and conductors, franchising of public utility vehicles and enforcing traffic rules and regulations and adjudicating apprehensions. On January 30, 1987, the Land Transportation Commission was abolished and two offices were created, namely: The Land Transportation Office (LTO) and the Land Transpor...<br>

hit_rate: 0.0
mrr: 0.0
precision: 0.0
recall: 0.0
ap: 0.0
ndcg: 0.0


Query: What significant event occurred on June 20, 1964, that affected the Motor Vehicle Office and what new entity was created as a result of this event?
Metrics: {'hit_rate': 0.0, 'mrr': 0.0, 'precision': 0.0, 'recall': 0.0, 'ap': 0.0, 'ndcg': 0.0}



In [23]:
evaluator = HybridRetrievalEvaluator.from_metric_names(
    metric_names=metrics,
    faiss_retriever=faiss_retriever,
    bm25_retriever=bm25_retriever,
    Print_Results=False  
)

eval_results = await evaluator.aevaluate_dataset(qa_dataset)

def display_results(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    columns = {
        "retrievers": [name],
        **{k: [full_df[k].mean()] for k in metrics},
    }

    metric_df = pd.DataFrame(columns)

    return metric_df

display_results("top-2 eval", eval_results)


hit_rate: 1.0
mrr: 0.25
precision: 0.06666666666666667
recall: 1.0
ap: 0.25
ndcg: 0.43067655807339306
hit_rate: 0.0
mrr: 0.0
precision: 0.0
recall: 0.0
ap: 0.0
ndcg: 0.0
hit_rate: 1.0
mrr: 0.3333333333333333
precision: 0.06666666666666667
recall: 1.0
ap: 0.3333333333333333
ndcg: 0.5
hit_rate: 1.0
mrr: 0.5
precision: 0.06666666666666667
recall: 1.0
ap: 0.5
ndcg: 0.6309297535714575
hit_rate: 0.0
mrr: 0.0
precision: 0.0
recall: 0.0
ap: 0.0
ndcg: 0.0
hit_rate: 1.0
mrr: 0.5
precision: 0.06666666666666667
recall: 1.0
ap: 0.5
ndcg: 0.6309297535714575
hit_rate: 0.0
mrr: 0.0
precision: 0.0
recall: 0.0
ap: 0.0
ndcg: 0.0
hit_rate: 1.0
mrr: 0.5
precision: 0.06666666666666667
recall: 1.0
ap: 0.5
ndcg: 0.6309297535714575
hit_rate: 1.0
mrr: 0.3333333333333333
precision: 0.06666666666666667
recall: 1.0
ap: 0.3333333333333333
ndcg: 0.5
hit_rate: 1.0
mrr: 0.5
precision: 0.06666666666666667
recall: 1.0
ap: 0.5
ndcg: 0.6309297535714575
hit_rate: 1.0
mrr: 0.5
precision: 0.06666666666666667
recall: 1.0
ap: 

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,top-2 eval,0.833333,0.422829,0.055556,0.833333,0.422829,0.522589


# 6. Post Retrieval

## 6.A Summarization

In [24]:
def summarize_each_chunk(nodes, client, query, model="llama3.3", parent=False):
    if parent:
        chunks = [doc.text for doc in nodes]
    else:
        chunks = [doc.node.text for doc in nodes]
    summaries = []
    
    for i, chunk in enumerate(chunks):
        prompt = f"""
        Summarize the following text in one concise paragraph, focusing on key points relevant to the query: "{query}".
        
        - Emphasize information directly related to the query.
        - Exclude unrelated, redundant, or speculative details.
        - Do NOT introduce new information or answer the query itself. 
        
        Text:
        {chunk}
        
        Summary:
        """
        
        response = client.generate(model=model, prompt=prompt)
        summary = response['response'].strip()
        summaries.append(summary)

    return summaries

# 6.B Evaluation Generation

In [25]:
def generate_response_with_notice(summaries, query, client, model="llama3.3"):
    # Combine summaries into context block
    context = "\n".join(summaries)
    
    # Create prompt to answer based on summarized text
    prompt = f"""
    Use the following summarized information to answer the query accurately and concisely. 
    DO NOT USE BACKGROUND KNOWLEDGE OUTSIDE THE CONTEXT PROVIDED.
    If the information is not sufficient to fully address the query, respond ONLY with:
    "The available information is insufficient to provide a complete answer to this query."

    Summarized Context:
    {context}
    
    Query:
    {query}
    
    Response:
    """
    
    # Send the prompt to Ollama
    response = client.generate(
        model=model,
        prompt=prompt
    )
    
    return response['response'].strip()

# 7. Querying

## 7.A Query Transforms

# 8. Query Generation

In [26]:
docstore = {}

# Store documents using full metadata as the key
for doc in documents:
    key = tuple(doc.metadata.items())  # Convert metadata to tuple for hashable key
    docstore[key] = doc

In [27]:
def get_document_by_chunk_metadata(chunk_node):
    # Convert chunk metadata to tuple for matching
    metadata_key = tuple(chunk_node.metadata.items())

    # Retrieve document from docstore
    document = docstore.get(metadata_key)
    return document

In [28]:
def remove_duplicate_documents(doc_list):
    seen_ids = set()
    unique_docs = []

    for doc in doc_list:
        if doc.doc_id not in seen_ids:
            seen_ids.add(doc.doc_id)
            unique_docs.append(doc)

    return unique_docs

In [29]:
def gen_query(query, top_k, client, mode='dense', summary=False, model="llama3.3", chunks_only=False):
    response = client.embeddings(prompt=query, model="mxbai-embed-large")
    query_embedding = response["embedding"]

    top_k_docs = faiss_retriever._retrieve(query_embedding, top_k=top_k)

    bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=top_k,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
    )
    retrieved_nodes = bm25_retriever.retrieve(query)

    results = {'faiss': top_k_docs, 'bm25':retrieved_nodes}
    ranked_results = hybrid_embedding(results, top_k=top_k)

    if mode == 'dense':
        print('using FAISS')
        ans_nodes =top_k_docs
    elif mode == 'sparse':
        print('using BM25')
        ans_nodes = retrieved_nodes
    else:
        print('using Hybrid')
        ans_nodes = ranked_results

    parent_flag = True
    context = set([get_document_by_chunk_metadata(docs).text for docs in ans_nodes])
    if chunks_only:
        parent_flag = False
        print('using chunks only')
        context = [docs.node.text for docs in ans_nodes]
        
    if summary:
        print('using summaries')
        context_nodes = remove_duplicate_documents([get_document_by_chunk_metadata(docs) for docs in ans_nodes])

        if chunks_only:
            context_nodes=ans_nodes
        summaries = summarize_each_chunk(context_nodes, client, model='llama3.3', query=query,parent=parent_flag)
        context = summaries

    answer = generate_response_with_notice(context, query, client, model=model)

    # Format the references
    references = []
    for i, doc in enumerate(ranked_results[:top_k], start=1):
        metadata = doc.metadata
        source_info = f"Source {i}: {metadata['title']} (Page {metadata['page']}, Folder: {metadata['folder']})"
        references.append(source_info)

    return answer, "\n".join(references), "\n".join(context)

# 9. TDC Exam Evaluation

In [30]:
# Generate prompts dynamically
def generate_prompt(row):
    options = []
    for choice in ['A', 'B', 'C', 'D', 'E']:
        # Check for NaN or blank values
        if pd.notna(row[choice]) and row[choice] != '':
            options.append(f"{choice}. {row[choice]}")
    
    # Construct the prompt with few-shot examples
    prompt = f"\nActual Question: {row['Question']}\n" + "\n".join(options)
    prompt += "\nPlease answer only in letters and put them inside a bracket '[]'. If the question contains the statement 'Check all that apply' then add comma separator if there are multiple answers ONLY IF ALLOWED."
    
    return prompt

In [31]:
# Load the Excel file
file_path = '/mnt/c/Users/Jeryl Salas/Documents/AI 351/Project/LTO_EXAM.csv'
df = pd.read_csv(file_path)
df['Prompt'] = df.apply(generate_prompt, axis=1)
display(df.head())

,Question,A,B,C,D,E,Answer,Prompt
0,What should you do in case your vehicle breaks...,Open your trunk and hood,Stand on the expressway and flag down passing ...,Call for help using a mobile phone or an expre...,Park as far to the right as possible,Put your hazard warning light on,"A, C, D, E",\nActual Question: What should you do in case ...
1,What will happen when your front tire blows out?,The back end will sway towards the side of the...,The back end will sway away from the blowout,The front end will pull towards the side of th...,The front end will pull to the opposite side o...,NaN,C,\nActual Question: What will happen when your ...
2,What should you do when an ambulance comes up ...,Stop as soon as you can,"Maintain your speed, let the ambulance driver ...",Speed up so that you don't hold the ambulance,Pull over to the right and slow down or even s...,NaN,D,\nActual Question: What should you do when an ...
3,While driving the hood of your car lifts up bl...,Look through the gap underneath the hood or ou...,Brake suddenly so you don't leave the road,Pull to the side of the road and refasten the ...,Turn your headlights on and look out of the si...,NaN,"A,C",\nActual Question: While driving the hood of y...
4,"In case of an accident, the first duty of the ...",pick-up the injured person and take him to the...,report the accident to the nearest hospital,report the accident to the nearest police station,NaN,NaN,A,"\nActual Question: In case of an accident, the..."


In [ ]:
qr_range = (0,60)
df["AI"] = np.nan
ai_answer = []
for i in tqdm(range(*qr_range)):
    ai_answer.append(gen_query(df.loc[i,"Prompt"], top_k=15, client=client, mode='hybrid', model="llama3.3"))

df.loc[qr_range[0]:qr_range[1]-1, "AI"] = [answ[0] for answ in ai_answer]
df.loc[qr_range[0]:qr_range[1]-1, "Context"] = [answ[2] for answ in ai_answer]

  0%|          | 0/60 [00:00<?, ?it/s]

using Hybrid


  2%|▏         | 1/60 [00:40<39:40, 40.34s/it]

using Hybrid


  3%|▎         | 2/60 [01:18<37:48, 39.11s/it]

...

 62%|██████▏   | 37/60 [24:00<15:33, 40.57s/it]

using Hybrid


 63%|██████▎   | 38/60 [24:42<14:58, 40.83s/it]

using Hybrid


 65%|██████▌   | 39/60 [25:22<14:16, 40.77s/it]

In [ ]:
import re


def process_answers(answers):
    formatted_answers = []
    
    for a in answers:
        
        matches = re.findall(r'\[?\s*([A-E](?:\s*,\s*[A-E])*)\s*\]?', str(a)) # Extract answers like [A, C, D] or [A] or [B, D]
        answers = []
        for match in matches:
            answers.extend(re.split(r'\s*,\s*', match))  # Split by comma and remove spaces
        unique_sorted_answers = sorted(set(answers), key=lambda x: ['A', 'B', 'C', 'D', 'E'].index(x))
        if not unique_sorted_answers:
            formatted_answers.append(None)
        else:
            formatted_answers.append(unique_sorted_answers)
    return formatted_answers

df_results = df.loc[qr_range[0]:qr_range[1]-1, ["Question","Answer","AI"]]
df_results['Answer'] = df_results['Answer'].apply(lambda x: x.split(', '))
df_results['AI'] = process_answers(df_results["AI"])
df_results['Answer'] = process_answers(df_results["Answer"])



def calculate_scores(df):
    scores = []
    for index, row in df.iterrows():
        correct_answers = set(row['Answer'] if row['Answer'] is not None else [])
        ai_answers = set(row['AI'] if row['AI'] is not None else [])
        if ai_answers == correct_answers:
            score = 1.0
        else:
            score = 0.0
        scores.append(score)
    
    df['Score'] = scores
    accuracy = scores.count(1.0) / len(scores)
    print(f'Final Score: {scores.count(1.0):.2f}/{len(scores):.2f}')
    print(f'Accuracy: {accuracy:.2f}%')
    return df

# Apply the scoring function
scored_df = calculate_scores(df_results)

# Display the dataframe to verify the results
display(scored_df[['Question', 'Answer', 'AI', 'Score']])

Final Score: 33.00/60.00
Accuracy: 0.55%


,Question,Answer,AI,Score
0,What should you do in case your vehicle breaks...,"[A, C, D, E]","[C, D, E]",0.0
1,What will happen when your front tire blows out?,[C],[B],0.0
2,What should you do when an ambulance comes up ...,[D],[A],0.0
3,While driving the hood of your car lifts up bl...,"[A, C]","[A, C, D]",0.0
4,"In case of an accident, the first duty of the ...",[A],[C],0.0
5,"When a vehicle starts to skid, what should the...",[B],"[B, C]",0.0
6,"In case of injuries caused by an accident, the...",[A],"[A, B]",0.0
7,What will happen when your rear tire blows out?,[B],[A],0.0
8,"When a vehicle is stalled or disabled, the dri...",[C],[C],1.0
9,If you are the first to arrive at the scene of...,[B],[B],1.0


In [ ]:
# Generate prompts dynamically
def generate_prompt(row):
    
    # Construct the prompt with few-shot examples
    prompt = f"\nActual Question: {row['Question']}\n" 
    prompt += "\nPlease answer the question based on the given context."
    
    return prompt

In [ ]:
# Load the Excel file
file_path = '/mnt/c/Users/Jeryl Salas/Documents/AI 351/Project/LTO_EXAM_QnA.csv'
df = pd.read_csv(file_path, encoding='ISO-8859-1')
df['Prompt'] = df.apply(generate_prompt, axis=1)
display(df.head())

,Question,Answer,Prompt
0,Traffic Jam can be prevented if you,Keep opposing lanes open,\nActual Question: Traffic Jam can be prevente...
1,When making a right turn you should,Stay on the outermost lane of the road then si...,\nActual Question: When making a right turn yo...
2,"When you intend to turn right or left, signal ...",25 meters before you intend to make your turn,\nActual Question: When you intend to turn rig...
3,"At an intersection with a traffic light, make ...",The green light is on and there is a left turn...,\nActual Question: At an intersection with a t...
4,Graft and corruption in the traffic enforcemen...,Self disciplined by drivers and obeying traffi...,\nActual Question: Graft and corruption in the...


In [ ]:
qr_range = (0,60)
df["AI"] = np.nan
ai_answer = []
for i in tqdm(range(*qr_range)):
    ai_answer.append(gen_query(df.loc[i,"Prompt"], top_k=15, client=client, mode='hybrid', model="llama3.3"))

df.loc[qr_range[0]:qr_range[1]-1, "AI"] = [answ[0] for answ in ai_answer]
df.loc[qr_range[0]:qr_range[1]-1, "Context"] = [answ[2] for answ in ai_answer]
df_new = df.loc[qr_range[0]:qr_range[1]-1].copy()
df = df_new.copy()

  0%|          | 0/60 [00:00<?, ?it/s]

using Hybrid


  2%|▏         | 1/60 [00:53<53:00, 53.91s/it]

using Hybrid


  3%|▎         | 2/60 [01:35<45:18, 46.88s/it]

...

using Hybrid


  7%|▋         | 4/60 [02:55<39:16, 42.08s/it]

using Hybrid


  8%|▊         | 5/60 [03:50<42:49, 46.72s/it]

using Hybrid


# 10. Similarity Evaluation

In [ ]:
from llama_index.core.evaluation import SemanticSimilarityEvaluator
from llama_index.core.base.embeddings.base import BaseEmbedding
import asyncio
from llama_index.core.embeddings import resolve_embed_model
from pydantic import PrivateAttr

class OllamaEmbeddingModel(BaseEmbedding):
    _client: Client = PrivateAttr()

    def __init__(self, model_name: str = "mxbai-embed-large", timeout: int = 300):
        super().__init__()
        self.model_name = model_name
        self._client = Client() 

    async def _aget_query_embedding(self, query: str) -> list[float]:
        return await self._aget_text_embedding(query)

    async def _aget_text_embedding(self, text: str) -> list[float]:
        loop = asyncio.get_event_loop()
        embedding_response = await loop.run_in_executor(
            None, self._client.embeddings, self.model_name, text
        )
        return embedding_response['embedding']  

    def _get_query_embedding(self, query: str) -> list[float]:
        return self._get_text_embedding(query)

    def _get_text_embedding(self, text: str) -> list[float]:
        embedding_response = self._client.embeddings(
            model=self.model_name,
            prompt=text
        )
        return embedding_response['embedding']


embed_model = OllamaEmbeddingModel(model_name="mxbai-embed-large")
evaluator = SemanticSimilarityEvaluator(
    embed_model=embed_model,
    similarity_threshold=0.6
)

results_scores = []
results_passing = []
for i in tqdm(range(len(df))):
    response = df.loc[i, "AI"]
    reference = df.loc[i, "Answer"]

    result = await evaluator.aevaluate(
    response=response,
    reference=reference,
    )
    results_scores.append(result.score)
    results_passing.append(result.passing)
    
df['Score'] = results_scores
df['Passing'] = results_passing

average_score = df['Score'].mean()
total_items = len(df)
passing_items = df['Passing'].sum()  
print(f"Average Score: {average_score:.4f}")
print(f"Passing: {passing_items}/{total_items}")
display(df[['Question', 'Answer', 'AI', 'Score', 'Passing']])

  0%|          | 0/60 [00:00<?, ?it/s]

100%|██████████| 60/60 [00:23<00:00,  2.58it/s]

Average Score: 0.6398
Passing: 45/60


,Question,Answer,AI,Score,Passing
0,Traffic Jam can be prevented if you,Keep opposing lanes open,The actual question is not directly stated in ...,0.625760,True
1,When making a right turn you should,Stay on the outermost lane of the road then si...,"According to the given text, when making a rig...",0.760502,True
2,"When you intend to turn right or left, signal ...",25 meters before you intend to make your turn,"According to Article IV, Section 45 of the Tra...",0.663455,True
3,"At an intersection with a traffic light, make ...",The green light is on and there is a left turn...,"Based on the provided information, at an inter...",0.700548,True
4,Graft and corruption in the traffic enforcemen...,Self disciplined by drivers and obeying traffi...,The correct answer is:\n\nUsing a digitalized ...,0.614937,True
5,"On a four(4) lane road with single white line,...",Overtake by passing over the solid white line,"According to the provided text, on a multi-lan...",0.756958,True
6,A double solid yellow line with broken white l...,Absolutely no overtaking,"According to the provided text, a double solid...",0.607177,True
7,"When making a U-Turn, you should",Check for traffic behind you and indicate your...,"When making a U-turn, you should start turning...",0.765110,True
8,Signs that are triangular in shape and with a ...,Caution or warning signs,Horizontal Alignment Markers.,0.550258,False
9,"Signs that are round, inverted triangle or oct...",Regulatory signs,"Based on the provided text, it appears to be a...",0.681984,True


# 11. Relevancy Evaluation

In [ ]:
from llama_index.core.evaluation import RelevancyEvaluator

ollama_llm = Ollama(model="llama3.3", request_timeout=300)
evaluator = RelevancyEvaluator(llm=ollama_llm)

eval_results = []

for i in tqdm(range(len(df))):
    eval_result = await evaluator.aevaluate(
        query=df.loc[i, "Question"],
        response=df.loc[i, "AI"],
        contexts=[df.loc[i, "Context"]]  
    )
    eval_results.append(eval_result.passing)

df['Eval'] = eval_results

total_items = len(df)
passing_items = df['Eval'].sum()
score = f"Score: {passing_items}/{total_items}"
percentage = passing_items / total_items if total_items > 0 else 0
print(score)
print(f"Percentage: {percentage:.2%}")
display(df[['Question', 'Answer', 'AI', 'Eval']])

100%|██████████| 60/60 [30:32<00:00, 30.55s/it]

Score: 50/60
Percentage: 83.33%


,Question,Answer,AI,Eval
0,Traffic Jam can be prevented if you,Keep opposing lanes open,The actual question is not directly stated in ...,True
1,When making a right turn you should,Stay on the outermost lane of the road then si...,"According to the given text, when making a rig...",True
2,"When you intend to turn right or left, signal ...",25 meters before you intend to make your turn,"According to Article IV, Section 45 of the Tra...",True
3,"At an intersection with a traffic light, make ...",The green light is on and there is a left turn...,"Based on the provided information, at an inter...",True
4,Graft and corruption in the traffic enforcemen...,Self disciplined by drivers and obeying traffi...,The correct answer is:\n\nUsing a digitalized ...,True
5,"On a four(4) lane road with single white line,...",Overtake by passing over the solid white line,"According to the provided text, on a multi-lan...",False
6,A double solid yellow line with broken white l...,Absolutely no overtaking,"According to the provided text, a double solid...",True
7,"When making a U-Turn, you should",Check for traffic behind you and indicate your...,"When making a U-turn, you should start turning...",False
8,Signs that are triangular in shape and with a ...,Caution or warning signs,Horizontal Alignment Markers.,True
9,"Signs that are round, inverted triangle or oct...",Regulatory signs,"Based on the provided text, it appears to be a...",True


# 12. Correctness Evaluation

In [ ]:
from llama_index.core.evaluation import CorrectnessEvaluator

evaluator = CorrectnessEvaluator(llm=ollama_llm)

eval_results = []
feedback_results = []

for i in tqdm(range(len(df))):
    eval_result = await evaluator.aevaluate(
        query=df.loc[i, "Question"],
        response=df.loc[i, "AI"],
        reference=[df.loc[i, "Context"]]  
    )
    eval_results.append(eval_result.score)
    feedback_results.append(eval_result.feedback)

df['Score'] = eval_results
df['Feedback'] = feedback_results

total_items = len(df)
passing_items = df['Score'].sum()
score = f"Score: {passing_items}/{total_items}"
percentage = passing_items / total_items if total_items > 0 else 0
print(score)
print(f"Percentage: {percentage:.2%}")
display(df[['Question', 'Answer', 'AI', 'Score', 'Feedback']])

                                            Question  \
0  What should you do in case your vehicle breaks...   
1   What will happen when your front tire blows out?   
2  What should you do when an ambulance comes up ...   
3  While driving the hood of your car lifts up bl...   
4  In case of an accident, the first duty of the ...   

                                                   A  \
0                           Open your trunk and hood   
1  The back end will sway towards the side of the...   
2                            Stop as soon as you can   
3  Look through the gap underneath the hood or ou...   
4  pick-up the injured person and take him to the...   

                                                   B  \
0  Stand on the expressway and flag down passing ...   
1       The back end will sway away from the blowout   
2  Maintain your speed, let the ambulance driver ...   
3         Brake suddenly so you don't leave the road   
4        report the accident to the nearest ho

  0%|          | 0/91 [00:00<?, ?it/s]


KeyError: 'Context'